# Prolific API basic walkthrough
Documentation: https://docs.prolific.co/docs/api-docs/public/

## Setup notebook

In [1]:
# import libraries
import json
import urllib
import requests
import pymongo as pm
import pandas as pd

In [2]:
# define session wrapper to conveniently access API
class SessionWithUrlBase(requests.Session):
    def __init__(self, url_base=None, *args, **kwargs):
        super(SessionWithUrlBase, self).__init__(*args, **kwargs)
        self.url_base = url_base

    def request(self, method, url, **kwargs):
        modified_url = urllib.parse.urljoin(self.url_base, url)

        return super(SessionWithUrlBase, self).request(method, modified_url, **kwargs)

In [3]:
# get prolific credentials
tok = pd.read_csv('prolificToken.txt', header = None).values[0][0]

# create session
s = SessionWithUrlBase(url_base='https://api.prolific.co/api/v1/')
s.headers.update({
    'Authorization': f'Token {tok}',
    'Content-Type': 'application/json'
})

### Use case 1: Create a new draft Prolific Study
To create a new study, we execute a POST request to `studies/`, specifying the following study parameters in the POST call. The parameter descriptions can be found [here](https://docs.prolific.co/docs/api-docs/public/#tag/Studies/paths/~1api~1v1~1studies~1/post).

The documentation for specifying participant eligibility is incomplete, but you can modify `reqs` to modify the eligibility requirements

In [4]:
reqs = s.get('eligibility-requirements/').json()
reqs = reqs['results']

# it seems like
for study in reqs[221]['attributes']:
    study['value'] = True

In [5]:
# creates new study on Prolific
new_study = s.post('studies/',
      data=json.dumps({
          'name': 'BACH test study',
          'internal_name': 'test_study_0',
          'description': 'I think it works.',
          'external_study_url': 'https://cogtoolslab.org:8864/index.html?' +
          'PROLIFIC_PID={{%PROLIFIC_PID%}}&STUDY_ID={{%STUDY_ID%}}&SESSION_ID={{%SESSION_ID%}}',
          'prolific_id_option': 'url_parameters',
          'completion_code': 'BACH_completion_code',
          'completion_option': 'url',
          'total_available_places': 1,
          'estimated_completion_time': 10,
          'device_compatibility': ['desktop'],
          'reward': 200,
          'peripheral_requirements': [],
          'eligibility_requirements': reqs
      })
).json()

In [6]:
# display information about a specific study
study_inf = s.get(f"studies/{new_study['id']}/").json()
study_inf['total_available_places']

1

In [7]:
# Update study parameters. In this case, we increment study places by one
new_study = s.patch(f"studies/{new_study['id']}/",
        data= json.dumps({
            'total_available_places': new_study['total_available_places'] + 1,
        })
).json()
new_study['total_available_places']

2

In [8]:
# delete study
s.delete(f"studies/{new_study['id']}/")

<Response [204]>

watchdog integration

document so that someone else can define what their study is in prolific and publish it

test case: object localization - see if someone can run 2 pilot participants on the object localization study